In [8]:
import pandas as pd
import numpy as np
import gcsfs
import sys
import xarray as xr
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/tmp/ipykernel_268429/1529845526.py:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [9]:
df = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores.csv')

In [10]:
# define a simple search on keywords
def search_df(df, verbose= False, **search):
    "search by keywords - if list, then match exactly, otherwise match as substring"
    keys = ['activity_id','institution_id','source_id','experiment_id','member_id', 'table_id', 'variable_id', 'grid_label']
    d = df
    for skey in search.keys():
        
        if isinstance(search[skey], str):  # match a string as a substring
            d = d[d[skey].str.contains(search[skey])]
        else:
            dk = []
            for key in search[skey]:       # match a list of strings exactly
                dk += [d[d[skey]==key]]
            d = pd.concat(dk)
            keys.remove(skey)
    if verbose:
        for key in keys:
            print(key,' = ',list(d[key].unique()))      
    return d

In [11]:
#dfp = search_df(df, experiment_id=['historical','ssp585'], table_id=['Omon','Amon'], variable_id=["uas","vas","tos"], grid_label=['gr'])
dfp = search_df(df,
                experiment_id=["historical","ssp585","ssp245"],
                table_id=["Amon","Amon", "Amon", "Amon", "Omon", "SImon", "SImon", "SImon", "SImon","Amon","Omon"],
                variable_id=["prw","clt", "uas", "vas", "chl", "sithick", "siconc", "sisnthick", "sisnconc", "tas","tos"],
                grid_label=['gn'])
#dfp = search_df(df, experiment_id=['historical','ssp585'], member_id=['r1i1p1f1'],table_id=['Amon','SImon','Omon','Amon'], variable_id=["uas","vas","chl","clt"], grid_label=['gn'])


In [12]:
dm = dfp[["experiment_id","source_id","variable_id","member_id","grid_label"]].groupby(["member_id","source_id","experiment_id","grid_label"]).nunique()[["variable_id"]]

In [13]:
#dm2=dm[dm["experiment_id"] in dm["historical","ssp585"]]
#print(dm2)
table = pd.DataFrame.pivot_table(dm, values="variable_id", index=["source_id","member_id"],
                                 columns=["grid_label","experiment_id"], aggfunc=np.sum, fill_value=0)

table[(table.values.sum(axis=1)==33)]

Empty DataFrame
Columns: [(gr, historical), (gr, ssp245), (gr, ssp585)]
Index: []

In [14]:
#table[(table.uas>0)&(table.vas>0)&(table.chl>0)&(table.clt>0)&(table.sisnconc>0)&(table.sisnthick>0)&(table.siconc>0)&(table.sithick>0)]
table[(table.sithick>0)&(table.siconc>0)&(table.sisnconc>0)&(table.sisnthick>0)]
print(dm)


AttributeError: 'DataFrame' object has no attribute 'sithick'